In [4]:
#read the distance and location files and create a panda dataframe for both
import os
import pandas as pd

project_root = os.path.abspath(os.getcwd())

data_dir = os.path.join(project_root, '..', 'data')

location_EV = pd.read_csv(os.path.join(data_dir, 'location_detail.csv'), sep=',')

distance_driven_EV = pd.read_csv(os.path.join(data_dir, 'distance_driven.csv'), sep=',')



In [6]:
print(location_EV.head())   
print(distance_driven_EV.head())

   Unnamed: 0     0   1   2   3   4   5   6   7   8  ... 1991 1992 1993 1994  \
0           0  0.00  ho  ho  ho  ho  ho  ho  ho  ho  ...   ho   ho   ho   ho   
1           1  0.25  ho  ho  ho  ho  ho  ho  ho  ho  ...   ho   ho   ho   ho   
2           2  0.50  ho  ho  ho  ho  ho  ho  ho  ho  ...   ho   ho   ho   ho   
3           3  0.75  ho  ho  ho  ho  ho  ho  ho  ho  ...   ho   ho   ho   ho   
4           4  1.00  ho  ho  ho  ho  ho  ho  ho  ho  ...   ho   ho   ho   ho   

  1995 1996 1997 1998 1999 2000  
0   ho   ho   ho   ho   ho   ho  
1   ho   ho   ho   ho   ho   ho  
2   ho   ho   ho   ho   ho   ho  
3   ho   ho   ho   ho   ho   ho  
4   ho   ho   ho   ho   ho   ho  

[5 rows x 2002 columns]
   Unnamed: 0      0     1      2      3      4      5      6      7      8  \
0           0  155.0  54.0  203.0  149.0  138.0  116.0  161.0  180.0  311.0   

   ...  1990   1991   1992   1993   1994  1995  1996   1997   1998  1999  
0  ...  67.0  405.0  472.0  391.0  300.0  92.0  38.0  15

In [ ]:
case_number = 7 
# Select the time column and the column "6" from both dataframes
time = location_EV.iloc[:, 1]  # Assuming the second column is time (first is Unnamed: 0)
location_col = location_EV.iloc[:, case_number]
distance_driven_case = distance_driven_EV.iloc[:, case_number]

# Combine into a single dataframe
ev_data = pd.DataFrame({
    'Time': time,
    'Location': location_col,
})



In [21]:
#count the amount of "dr" in the location column
dr_count = location_col.str.count('dr').sum()
print(f"Number of 'dr' in the location column: {dr_count}")


Number of 'dr' in the location column: 29


In [22]:
#add a new column called "distance driven" to the dataframe and fill it with 0 except for the rows where the location column constains "dr"
# there, distance_driven_case / dr_count
ev_data['Distance Driven'] = 0
ev_data.loc[location_col.str.contains('dr'), 'Distance Driven'] = distance_driven_case / dr_count



In [23]:
#add a new column called "EV_at_home" with 0 if car is not at home and 1 if car is at home
ev_data['EV_at_home'] = 0
ev_data.loc[location_col.str.contains('home'), 'EV_at_home'] = 1

In [24]:
#to csv
ev_data.to_csv(os.path.join(data_dir, 'ev_data_combined.csv'), index=False)